In [11]:
import math
import pandas as pd
import glob
import numpy as np

def radiance(alpha:"nm"):
    new_alpha = alpha * 10**(-9) # nm -> m
    c = 2.99792458 * 10**8
    h = 6.6260755 * 10**(-34)
    k = 1.380658 * 10**(-23)
    T = 6504
    return (c**2*2*h)/(new_alpha**5*(math.exp((h*c)/(k*T*new_alpha))-1))

###  CIEXYZ1931 coordinate & sRGB

In [33]:
Color_Matching_Functions = pd.read_csv("./XYZ_CIE_2.dat", sep = "\t", header=None, index_col=0)

M = np.array([[3.2406, -1.5372, -0.4986],
            [-0.9689, 1.8758, 0.0415],
            [0.0557, -0.2040, 1.0570]])

for file in glob.glob(r".\Pigmentdata\*"):
    filename = file.split('\\')[-1]
    df = pd.read_csv(file, sep = "\t", header=None)
    start_value = df[1][0]
    finish_value = df[1][255]
    Xcount = 0
    Ycount = 0
    Zcount = 0
    for j in range(380,400):
        Xcount += Color_Matching_Functions[1][j]*start_value*radiance(j)
        Ycount += Color_Matching_Functions[2][j]*start_value*radiance(j)
        Zcount += Color_Matching_Functions[3][j]*start_value*radiance(j)
        
    for i in range(df.shape[0]):
        key = int(round(df[0][i]))
        Xcount += Color_Matching_Functions[1][key]*df[1][i]*radiance(key)
        Ycount += Color_Matching_Functions[2][key]*df[1][i]*radiance(key)
        Zcount += Color_Matching_Functions[3][key]*df[1][i]*radiance(key)
        
    for n in range(700, 781):
        Xcount += Color_Matching_Functions[1][j]*finish_value*radiance(j)
        Ycount += Color_Matching_Functions[2][j]*finish_value*radiance(j)
        Zcount += Color_Matching_Functions[3][j]*finish_value*radiance(j)
        
    X = Xcount/(Xcount + Ycount + Zcount)
    Y = Ycount/(Xcount + Ycount + Zcount)
    Z = Zcount/(Xcount + Ycount + Zcount)
    # print(filename, X, Y)
    
    # sRGB
    XYZ = np.array([X, Y, Z]).reshape(-1, 1)
    RGB = M @ XYZ
    for clo in range(3):
        if RGB[clo][0]>1:
            RGB[clo][0] = 1
            
        if RGB[clo][0]<0:
            RGB[clo][0]=0
            
    # print(RGB.flatten())
    
    sRGB = np.zeros((3,1))
    Product = 1
    for clo0 in range(3):
        if RGB[clo0][0] < 0.0031308:
            sRGB[clo0][0] = 12.92*RGB[clo0][0]
        else:
            sRGB[clo0][0] = RGB[clo0][0]**(1.0/2.4)*1.055-0.055
    
    # print(sRGB.flatten())
    dis  = 0
    for clo3 in range(3):
        sRGB[clo3][0] = round(255*sRGB[clo3][0])
        dis += (255 - sRGB[clo3][0])**2
    
    print(filename, X, Y, sRGB.flatten().astype(int),dis)

15plot 0.4513247116745829 0.21269522038714805 [251   0 157] 74645.0
1plot 0.5203426369980633 0.3151198584053089 [255  86 104] 51362.0
33plot 0.3100577333732556 0.31621554688517584 [156 151 159] 29833.0
41plot 0.31886316834305994 0.3274481683306163 [160 153 154] 29630.0
46plot 0.31439422831893704 0.32150183832632395 [158 152 157] 29622.0
51plot 0.1733826268402902 0.11263917533382907 [ 51  76 223] 74681.0
58plot 0.19850816428404275 0.18510948417473133 [ 64 118 207] 57554.0
64plot 0.2908593619936975 0.2138485529717013 [163 105 187] 35588.0
6plot 0.4874920452532383 0.27620311417666105 [255  67 129] 51220.0
72plot 0.22905540022082344 0.389364552886566 [  0 192 157] 78598.0
74plot 0.26256092901860373 0.42682114628899664 [ 56 197 138] 56654.0
84plot 0.4873152708159332 0.4259264814878882 [241 156  50] 52022.0
92plot 0.44708788076439904 0.4629435186569031 [217 177  44] 52049.0
